# Model Pricing Analysis: Minion vs Standard Models

This notebook analyzes and visualizes the pricing differences between Minion and Standard model configurations.

## Overview
- **Minion Models**: Small, efficient models primarily from HuggingFace for domain-specific tasks
- **Standard Models**: High-performance models from major providers for complex tasks

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display, HTML

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## Data Configuration

### Minion Models (from domain_mappings.py)
Domain-specific models optimized for cost-efficiency

In [ ]:
# Minion models from adaptive_ai/config/domain_mappings.py
minion_models = {
    "meta-llama/Llama-3.1-8B-Instruct": {"provider": "huggingface"},
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B": {"provider": "huggingface"},
    "mistralai/Mistral-7B-Instruct-v0.3": {"provider": "huggingface"},
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B": {"provider": "huggingface"},
}

print("Minion Models:")
for model in minion_models.keys():
    print(f"  - {model}")

### Standard Models (from task_mappings.py)
High-performance models for complex reasoning and generation tasks

In [ ]:
# Standard models from adaptive_ai/config/task_mappings.py (unique models)
standard_models = {
    "gpt-4o": {"provider": "openai"},
    "claude-3-5-sonnet-20241022": {"provider": "anthropic", "mapped_name": "claude-4-sonnet"},
    "grok-3": {"provider": "grok"},
    "deepseek-chat": {"provider": "deepseek"},
    "gpt-4o-mini": {"provider": "openai"},
    "claude-3-haiku-20240307": {"provider": "anthropic", "mapped_name": "claude-3.5-haiku"},
    "grok-3-mini": {"provider": "grok"},
    "llama-3.1-70b-versatile": {"provider": "groq", "mapped_name": "llama-3.3-70b-versatile"},
}

print("Standard Models:")
for model in standard_models.keys():
    print(f"  - {model}")

### Pricing Data
Extracted from `seed-providers.ts` and `providers.py`

In [ ]:
# Pricing data from seed-providers.ts and providers.py (cost per 1M tokens in USD)
pricing_data = {
    # OpenAI models
    "gpt-4o": {"input": 2.50, "output": 10.00},
    "gpt-4o-mini": {"input": 0.15, "output": 0.60},
    
    # Anthropic models
    "claude-4-sonnet": {"input": 3.00, "output": 15.00},
    "claude-3.5-haiku": {"input": 0.80, "output": 4.00},
    
    # DeepSeek models
    "deepseek-chat": {"input": 0.14, "output": 0.28},
    
    # Grok models
    "grok-3": {"input": 3.00, "output": 15.00},
    "grok-3-mini": {"input": 0.30, "output": 0.50},
    
    # Groq models
    "llama-3.3-70b-versatile": {"input": 0.59, "output": 0.79},
    
    # HuggingFace models (all have same pricing in seed-providers.ts)
    "meta-llama/Llama-3.1-8B-Instruct": {"input": 0.01, "output": 0.02},
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B": {"input": 0.01, "output": 0.02},
    "mistralai/Mistral-7B-Instruct-v0.3": {"input": 0.01, "output": 0.02},
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B": {"input": 0.01, "output": 0.02},
}

print("Sample pricing data:")
for model, price in list(pricing_data.items())[:3]:
    print(f"  {model}: Input ${price['input']}, Output ${price['output']}")

## Data Processing Functions

In [ ]:
def get_model_pricing(model_name, model_config):
    """Get pricing for a model, handling name mappings."""
    # Check if there's a mapped name
    lookup_name = model_config.get("mapped_name", model_name)
    
    if lookup_name in pricing_data:
        return pricing_data[lookup_name]
    elif model_name in pricing_data:
        return pricing_data[model_name]
    else:
        return {"input": 0, "output": 0}  # Not found

def extract_pricing_data():
    """Extract pricing data for minion and standard models."""
    
    # Extract minion pricing
    minion_data = []
    for model, config in minion_models.items():
        pricing = get_model_pricing(model, config)
        minion_data.append({
            "model": model.split("/")[-1] if "/" in model else model,
            "full_model": model,
            "provider": config["provider"],
            "input_cost": pricing["input"],
            "output_cost": pricing["output"],
            "category": "Minion"
        })
    
    # Extract standard pricing
    standard_data = []
    for model, config in standard_models.items():
        pricing = get_model_pricing(model, config)
        standard_data.append({
            "model": model,
            "full_model": model,
            "provider": config["provider"],
            "input_cost": pricing["input"],
            "output_cost": pricing["output"],
            "category": "Standard"
        })
    
    return pd.DataFrame(minion_data), pd.DataFrame(standard_data)

# Extract the data
minion_df, standard_df = extract_pricing_data()

print("Data extraction complete!")
print(f"Minion models: {len(minion_df)}")
print(f"Standard models: {len(standard_df)}")

## Data Overview

In [ ]:
print("=" * 80)
print("MINION MODELS PRICING")
print("=" * 80)
display(minion_df[['model', 'provider', 'input_cost', 'output_cost']])

print("\n" + "=" * 80)
print("STANDARD MODELS PRICING")
print("=" * 80)
display(standard_df[['model', 'provider', 'input_cost', 'output_cost']])

## Visualization 1: Separate Input and Output Cost Charts

### Minion Models - Input vs Output Costs

In [ ]:
# Create figure with subplots for minion models
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
fig.suptitle('Minion Models - Input vs Output Costs (per 1M tokens)', fontsize=16, fontweight='bold')

minion_color = '#2E8B57'  # Sea Green

# Input costs
bars1 = ax1.bar(range(len(minion_df)), minion_df['input_cost'], 
                color=minion_color, alpha=0.7, edgecolor='black', linewidth=1)
ax1.set_title('Input Costs', fontsize=14, fontweight='bold')
ax1.set_xlabel('Models', fontsize=12)
ax1.set_ylabel('Cost (USD)', fontsize=12)
ax1.set_xticks(range(len(minion_df)))
ax1.set_xticklabels(minion_df['model'], rotation=45, ha='right')
ax1.grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(minion_df['input_cost']):
    ax1.text(i, v + 0.0001, f'${v:.3f}', ha='center', va='bottom', fontweight='bold')

# Output costs
bars2 = ax2.bar(range(len(minion_df)), minion_df['output_cost'], 
                color=minion_color, alpha=0.7, edgecolor='black', linewidth=1)
ax2.set_title('Output Costs', fontsize=14, fontweight='bold')
ax2.set_xlabel('Models', fontsize=12)
ax2.set_ylabel('Cost (USD)', fontsize=12)
ax2.set_xticks(range(len(minion_df)))
ax2.set_xticklabels(minion_df['model'], rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(minion_df['output_cost']):
    ax2.text(i, v + 0.0001, f'${v:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### Standard Models - Input vs Output Costs

In [ ]:
# Create figure with subplots for standard models
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
fig.suptitle('Standard Models - Input vs Output Costs (per 1M tokens)', fontsize=16, fontweight='bold')

standard_color = '#4169E1'  # Royal Blue

# Input costs
bars1 = ax1.bar(range(len(standard_df)), standard_df['input_cost'], 
                color=standard_color, alpha=0.7, edgecolor='black', linewidth=1)
ax1.set_title('Input Costs', fontsize=14, fontweight='bold')
ax1.set_xlabel('Models', fontsize=12)
ax1.set_ylabel('Cost (USD)', fontsize=12)
ax1.set_xticks(range(len(standard_df)))
ax1.set_xticklabels(standard_df['model'], rotation=45, ha='right')
ax1.grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(standard_df['input_cost']):
    ax1.text(i, v + max(standard_df['input_cost']) * 0.01, f'${v:.2f}', 
            ha='center', va='bottom', fontweight='bold')

# Output costs
bars2 = ax2.bar(range(len(standard_df)), standard_df['output_cost'], 
                color=standard_color, alpha=0.7, edgecolor='black', linewidth=1)
ax2.set_title('Output Costs', fontsize=14, fontweight='bold')
ax2.set_xlabel('Models', fontsize=12)
ax2.set_ylabel('Cost (USD)', fontsize=12)
ax2.set_xticks(range(len(standard_df)))
ax2.set_xticklabels(standard_df['model'], rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate(standard_df['output_cost']):
    ax2.text(i, v + max(standard_df['output_cost']) * 0.01, f'${v:.2f}', 
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## Visualization 2: Direct Comparison - Minion vs Standard

In [ ]:
# Create combined comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
fig.suptitle('Direct Pricing Comparison: Minion vs Standard Models', fontsize=16, fontweight='bold')

# Combine data for comparison
all_models = []
input_costs = []
output_costs = []
categories = []

# Add minion models
for _, row in minion_df.iterrows():
    all_models.append(f"{row['model']}\n(Minion)")
    input_costs.append(row['input_cost'])
    output_costs.append(row['output_cost'])
    categories.append('Minion')

# Add standard models
for _, row in standard_df.iterrows():
    all_models.append(f"{row['model']}\n(Standard)")
    input_costs.append(row['input_cost'])
    output_costs.append(row['output_cost'])
    categories.append('Standard')

colors = ['#2E8B57' if cat == 'Minion' else '#4169E1' for cat in categories]

# Input costs comparison
bars1 = ax1.bar(range(len(all_models)), input_costs, 
               color=colors, alpha=0.7, edgecolor='black', linewidth=1)
ax1.set_title('Input Costs Comparison (per 1M tokens)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Models', fontsize=12)
ax1.set_ylabel('Cost (USD)', fontsize=12)
ax1.set_xticks(range(len(all_models)))
ax1.set_xticklabels(all_models, rotation=45, ha='right')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')  # Log scale to better show the difference

# Output costs comparison
bars2 = ax2.bar(range(len(all_models)), output_costs, 
               color=colors, alpha=0.7, edgecolor='black', linewidth=1)
ax2.set_title('Output Costs Comparison (per 1M tokens)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Models', fontsize=12)
ax2.set_ylabel('Cost (USD)', fontsize=12)
ax2.set_xticks(range(len(all_models)))
ax2.set_xticklabels(all_models, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')  # Log scale to better show the difference

# Add legends
minion_patch = plt.Rectangle((0,0),1,1, color='#2E8B57', alpha=0.7, label='Minion Models')
standard_patch = plt.Rectangle((0,0),1,1, color='#4169E1', alpha=0.7, label='Standard Models')
ax1.legend(handles=[minion_patch, standard_patch], loc='upper left')
ax2.legend(handles=[minion_patch, standard_patch], loc='upper left')

plt.tight_layout()
plt.show()

## Visualization 3: Cost Ratio Analysis

In [ ]:
# Create scatter plot showing input vs output costs
fig, ax = plt.subplots(figsize=(12, 8))

# Plot minion models
ax.scatter(minion_df['input_cost'], minion_df['output_cost'], 
          s=200, alpha=0.7, color='#2E8B57', label='Minion Models', edgecolors='black')

# Plot standard models
ax.scatter(standard_df['input_cost'], standard_df['output_cost'], 
          s=200, alpha=0.7, color='#4169E1', label='Standard Models', edgecolors='black')

# Add model labels
for _, row in minion_df.iterrows():
    ax.annotate(row['model'], (row['input_cost'], row['output_cost']), 
               xytext=(5, 5), textcoords='offset points', fontsize=8)

for _, row in standard_df.iterrows():
    ax.annotate(row['model'], (row['input_cost'], row['output_cost']), 
               xytext=(5, 5), textcoords='offset points', fontsize=8)

ax.set_xlabel('Input Cost (USD per 1M tokens)', fontsize=12)
ax.set_ylabel('Output Cost (USD per 1M tokens)', fontsize=12)
ax.set_title('Input vs Output Cost Analysis', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xscale('log')
ax.set_yscale('log')

plt.tight_layout()
plt.show()

## Statistical Summary

In [ ]:
# Calculate summary statistics
minion_stats = {
    'avg_input': minion_df['input_cost'].mean(),
    'avg_output': minion_df['output_cost'].mean(),
    'min_input': minion_df['input_cost'].min(),
    'max_input': minion_df['input_cost'].max(),
    'min_output': minion_df['output_cost'].min(),
    'max_output': minion_df['output_cost'].max(),
}

standard_stats = {
    'avg_input': standard_df['input_cost'].mean(),
    'avg_output': standard_df['output_cost'].mean(),
    'min_input': standard_df['input_cost'].min(),
    'max_input': standard_df['input_cost'].max(),
    'min_output': standard_df['output_cost'].min(),
    'max_output': standard_df['output_cost'].max(),
}

# Create summary table
summary_data = {
    'Metric': ['Average Input Cost', 'Average Output Cost', 'Min Input Cost', 'Max Input Cost', 'Min Output Cost', 'Max Output Cost'],
    'Minion Models': [f"${minion_stats['avg_input']:.3f}", f"${minion_stats['avg_output']:.3f}", 
                     f"${minion_stats['min_input']:.3f}", f"${minion_stats['max_input']:.3f}",
                     f"${minion_stats['min_output']:.3f}", f"${minion_stats['max_output']:.3f}"],
    'Standard Models': [f"${standard_stats['avg_input']:.2f}", f"${standard_stats['avg_output']:.2f}", 
                       f"${standard_stats['min_input']:.2f}", f"${standard_stats['max_input']:.2f}",
                       f"${standard_stats['min_output']:.2f}", f"${standard_stats['max_output']:.2f}"]
}

summary_df = pd.DataFrame(summary_data)

print("=" * 80)
print("PRICING SUMMARY ANALYSIS")
print("=" * 80)
display(summary_df)

# Calculate cost differences
input_ratio = standard_stats['avg_input'] / minion_stats['avg_input'] if minion_stats['avg_input'] > 0 else float('inf')
output_ratio = standard_stats['avg_output'] / minion_stats['avg_output'] if minion_stats['avg_output'] > 0 else float('inf')

print("\nCOST DIFFERENCE ANALYSIS:")
print("-" * 50)
print(f"Standard models cost {input_ratio:.1f}x more for input tokens")
print(f"Standard models cost {output_ratio:.1f}x more for output tokens")
print(f"Input cost difference: ${standard_stats['avg_input'] - minion_stats['avg_input']:.3f} per 1M tokens")
print(f"Output cost difference: ${standard_stats['avg_output'] - minion_stats['avg_output']:.3f} per 1M tokens")

## Cost Efficiency Analysis

Let's analyze the cost efficiency for a hypothetical workload:

In [ ]:
# Simulate costs for different workload scenarios
scenarios = {
    'Light Usage': {'input_tokens': 1_000_000, 'output_tokens': 500_000},
    'Medium Usage': {'input_tokens': 10_000_000, 'output_tokens': 5_000_000},
    'Heavy Usage': {'input_tokens': 100_000_000, 'output_tokens': 50_000_000}
}

def calculate_workload_cost(input_tokens, output_tokens, df):
    """Calculate total cost for a workload using average pricing."""
    avg_input_cost = df['input_cost'].mean()
    avg_output_cost = df['output_cost'].mean()
    
    total_cost = (avg_input_cost * input_tokens / 1_000_000) + (avg_output_cost * output_tokens / 1_000_000)
    return total_cost

# Calculate costs for each scenario
scenario_results = []
for scenario_name, workload in scenarios.items():
    minion_cost = calculate_workload_cost(workload['input_tokens'], workload['output_tokens'], minion_df)
    standard_cost = calculate_workload_cost(workload['input_tokens'], workload['output_tokens'], standard_df)
    savings = standard_cost - minion_cost
    savings_pct = (savings / standard_cost) * 100 if standard_cost > 0 else 0
    
    scenario_results.append({
        'Scenario': scenario_name,
        'Input Tokens': f"{workload['input_tokens']:,}",
        'Output Tokens': f"{workload['output_tokens']:,}",
        'Minion Cost': f"${minion_cost:.2f}",
        'Standard Cost': f"${standard_cost:.2f}",
        'Savings': f"${savings:.2f}",
        'Savings %': f"{savings_pct:.1f}%"
    })

scenario_df = pd.DataFrame(scenario_results)

print("\nWORKLOAD COST COMPARISON")
print("=" * 80)
display(scenario_df)

## Conclusion

### Key Findings:

1. **Cost Efficiency**: Minion models are significantly more cost-effective, with costs that are orders of magnitude lower than standard models.

2. **Uniform Pricing**: All minion models from HuggingFace have the same pricing structure ($0.01 input, $0.02 output per 1M tokens).

3. **High Variance in Standard Models**: Standard model pricing varies widely, from budget options like DeepSeek Chat to premium models like Claude and Grok.

4. **Use Case Optimization**: The system is designed to route simpler, domain-specific tasks to cost-efficient minion models while reserving expensive standard models for complex reasoning tasks.

### Recommendations:

- Use minion models for domain-specific, routine tasks to maximize cost efficiency
- Reserve standard models for complex reasoning, creative tasks, and critical applications
- Consider the 100x+ cost difference when designing workload routing strategies